### Getting Started

We will be using [boto3](https://aws.amazon.com/sdk-for-python/) the Python SDK for AWS. It allows you to create, update, and delete AWS resources from your Python scripts. This library is preinstalled in this SageMaker Jupyter notebook. If yoou want to install this locally you can follow the instruction [here](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html). 

We will be using Amazon Comprehend, this service provides natural language processing, topic modeling, and Custom Classification capabilities, enabling a broad range of applications that can analyze text.

* **Natural Language Processing**: Amazon Comprehend requests for Entity Recognition, Sentiment Analysis, Syntax Analysis, Key Phrase Extraction, and Language Detection.
* **Topic Modeling**: Topic Modeling identifies relevant terms or topics from a collection of documents stored in Amazon S3. It will identify the most common topics in the collection and organize them in groups and then map which documents belong to which topic. 
* **Custom Comprehend**: The Custom Classification and Entities APIs can train a custom NLP model to categorize text and extract custom entities.

In this workshop we are going to be get visualize sentiment of Yelp reviews from 2015 using a data set from the AWS Open Data Registry [Yelp Reviews NLP Fast.ai](https://registry.opendata.aws/fast-ai-nlp/). 

In [ ]:
!pip install boto3 --upgrade

### [Download Yelp Reviews](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-download-file.html) 

We will download the reviews from the Fast.ai NLP dataset available on the [AWS Open Data Registry](https://registry.opendata.aws/fast-ai-nlp/). We will be using the boto3 library to instatntiate an S3 bucket reference to download the Yelp reviews

In [ ]:
open_data_bucket = 'fast-ai-nlp'
s3 = boto3.resource('s3')

s3.Bucket(open_data_bucket).download_file('yelp_review_full_csv.tgz', 'yelp_review_full_csv.tgz')

### Untar Yelp Reviews

There are two `csv` files in the tarball. One is called `train.csv`, the other is `test.csv`. If we were to leverage this data set to build an new AI/ML model we would have cleaned and split data sets for both train and test. 

In [ ]:
!tar -xvzf yelp_review_full_csv.tgz

For those interested, The `readme.txt` file describes in more details the dataset.

In [ ]:
!cat yelp_review_full_csv/readme.txt

### View raw csv file

We will use [Pandas](https://pandas.pydata.org/) to read the csv and view the data set. You will notice the data contains 2 unnamed columns for rating and review. The rating is between 1-5 and the review is a free form text field.

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', -1)

df = pd.read_csv('yelp_review_full_csv/train.csv', header=None)
df.head(5)

Now that we have a reference to the Yelp reviews with a pandas DataFrame we can pass the first review to the Amazon Comprehend APIs. We will create a reference to the Amazon Comprhend service and call each API in the cells below.

In [ ]:
import boto3

comprehend = boto3.client(service_name='comprehend')

This is all that is needed to start using Amazon Comprehend in Python. The below cells will execute on the NLP features available in Amazon Comprehend.

### Sentiment Analysis

The Sentiment Analysis API returns the overall sentiment of a text (Positive, Negative, Neutral, or Mixed).

We will use `pandas` to take a section of the json response and convert into a DataFrame to better see the result in this notebook. The raw json will look like below:

```json
{
    'Sentiment': 'NEUTRAL', 
    'SentimentScore': {
        'Positive': 0.00525312777608633, 
        'Negative': 0.0001172995544038713, 
        'Neutral': 0.9946269392967224, 
        'Mixed': 2.5949941573344404e-06}, 
    'ResponseMetadata': {
        'RequestId': '7bebc984-9c38-4ddc-b041-4a0e5a061b6e', 
        'HTTPStatusCode': 200, 
        'HTTPHeaders': {
            'x-amzn-requestid': '7bebc984-9c38-4ddc-b041-4a0e5a061b6e', 
            'content-type': 'application/x-amz-json-1.1', 
            'content-length': '164', 
            'date': 'Wed, 22 Jan 2020 19:27:13 GMT'
        }, 
        'RetryAttempts': 0
    }
}
```

Feel free to experiment with your own text and see the results that are generated.

In [ ]:
df.columns = ['rating', 'review']
review = df.at[0, 'review']
review

In [ ]:
from pandas.io.json import json_normalize

sentiment = comprehend.detect_sentiment(Text = review, LanguageCode= 'en')

json_normalize(sentiment['SentimentScore'])

### Keyphrase Extraction

The Keyphrase Extraction API returns the key phrases or talking points and a confidence score to support that this is a key phrase. In the resulting `KeyPhrases` object you will see the `BeginOffset` and `EndOffset` of the key phrase, the `Text` itself and `Score`. The `Score` is the level of confidence that Amazon Comprehend has in the accuracy of the detection.

In [ ]:
keyPhrases = comprehend.detect_key_phrases(Text = review, LanguageCode = 'en')

json_normalize(keyPhrases['KeyPhrases'])

### Entity Recognition

The Entity Recognition API returns the named entities ("People," "Places," "Locations," etc.) that are automatically categorized based on the provided text. We will use the same `review` from the other API calls for continuity but feel free to experiment with your own text.

In [ ]:
namedEntities = comprehend.detect_entities(Text = review, LanguageCode = 'en')

json_normalize(namedEntities['Entities'])

### Language Detection

The Language Detection API automatically identifies text written in over 100 languages and returns the dominant language with a confidence score to support that a language is dominant.

In [ ]:
language = comprehend.detect_dominant_language(Text = review)

json_normalize(language['Languages'])

### Amazon Translate

We will introduce you to another AI service to show how you can translate the text of the review into another language and detect a language other than English. To do so, much like Amazon Comprehend you provide a reference to the API.

In [ ]:
translate = boto3.client('translate')

result_de = translate.translate_text(Text=review, SourceLanguageCode="en", TargetLanguageCode="de")

translated = result_de['TranslatedText']
print(translated)

In [ ]:
translated_language = comprehend.detect_dominant_language(Text = translated)

json_normalize(translated_language['Languages'])

### Comprehend Medical

Medical Named Entity and Relationship Extraction (NERe)

The Amazon Comprehend APIs are generalized models to be used in many areas. Amazon Comprehend Medical provides a more specific NLP model for medical information.

The Medical NERe API returns the medical information such as medication, medical condition, test, treatment and procedures (TTP), anatomy, and Protected Health Information (PHI). It also identifies relationships between extracted sub-types associated to Medications and TTP. There is also contextual information provided as entity “traits” (negation, or if a diagnosis is a sign or symptom). The table below shows the extracted information with relevant sub-types and entity traits.

To only extract PHI, you can use the Protected Health Information Data Identification (PHId) API.

We will use the sample medical note below for this demo.

In [ ]:
medical_note = '''
Sample Type / Medical Specialty: Lab Medicine - Pathology
Sample Name: Lung Biopsy Pathology Report
Description: Lung, wedge biopsy right lower lobe and resection right upper lobe. Lymph node, biopsy level 2 and 4 and biopsy level 7 subcarinal. PET scan demonstrated a mass in the right upper lobe and also a mass in the right lower lobe, which were also identified by CT scan.
(Medical Transcription Sample Report)
CLINICAL HISTORY: A 48-year-old smoker found to have a right upper lobe mass on chest x-ray and is being evaluated for chest pain. PET scan demonstrated a mass in the right upper lobe and also a mass in the right lower lobe, which were also identified by CT scan. The lower lobe mass was approximately 1 cm in diameter and the upper lobe mass was 4 cm to 5 cm in diameter. The patient was referred for surgical treatment.

SPECIMEN:
A. Lung, wedge biopsy right lower lobe
B. Lung, resection right upper lobe
C. Lymph node, biopsy level 2 and 4
D. Lymph node, biopsy level 7 subcarinal

FINAL DIAGNOSIS:
A. Wedge biopsy of right lower lobe showing: Adenocarcinoma, Grade 2, Measuring 1 cm in diameter with invasion of the overlying pleura and with free resection margin.
B. Right upper lobe lung resection showing: Adenocarcinoma, grade 2, measuring 4 cm in diameter with invasion of the overlying pleura and with free bronchial margin. Two (2) hilar lymph nodes with no metastatic tumor.
C. Lymph node biopsy at level 2 and 4 showing seven (7) lymph nodes with anthracosis and no metastatic tumor.
D. Lymph node biopsy, level 7 subcarinal showing (5) lymph nodes with anthracosis and no metastatic tumor.

COMMENT: The morphology of the tumor seen in both lobes is similar and we feel that the smaller tumor involving the right lower lobe is most likely secondary to transbronchial spread from the main tumor involving the right upper lobe. This suggestion is supported by the fact that no obvious vascular or lymphatic invasion is demonstrated and adjacent to the smaller tumor, there is isolated nests of tumor cells within the air spaces. Furthermore, immunoperoxidase stain for Ck-7, CK-20 and TTF are performed on both the right lower and right upper lobe nodule. The immunohistochemical results confirm the lung origin of both tumors and we feel that the tumor involving the right lower lobe is due to transbronchial spread from the larger tumor nodule involving the right upper lobe.
'''

#### Detect PHI

Inspects the clinical text for protected health information (PHI) entities and entity category, location, and confidence score on that information.

If you are looking just PHI data you can use this call and find where the PHI is detected in the text with the `BeginOffset` and `EndOffset`, the `Score` on how accurate it has been detected, and the `Type` of PHI detected.

In [ ]:
comprehend_medical = boto3.client('comprehendmedical')

phi_response = comprehend_medical.detect_phi(Text = medical_note)

json_normalize(phi_response['Entities'])

#### Detect Entities

The DetectEntities operation will detect medical entities in your text. It detects entities in the following categories:

* ANATOMY
* MEDICAL_CONDITION
* MEDICATION
* PROTECTED_HEALTH_INFORMATION
* TEST_TREATMENT_PROCEDURE


In [ ]:
entities_response = comprehend_medical.detect_entities(Text = medical_note)

json_normalize(entities_response['Entities'])

This workshop has given you more information about how you can leverage our NLP services in your applications without having to have a deep understanding of ML and provide great insight in your domains. If you would like to dive deeper into the other features of our AI services check out this [link](https://aws.amazon.com/machine-learning/ai-services/).